# Scenario Objects - OOP Approach to PVDEG

Author: Tobin Ford | tobin.ford@nrel.gov

2024
****

A simple object orented workflow walkthrough using pvdeg.

In [9]:
# if running on google colab, uncomment the next line and execute this cell to install the dependencies and prevent "ModuleNotFoundError" in later cells:
# !pip install pvdeg==0.3.3

In [1]:
import pvdeg
import os

In [2]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("pvdeg version ", pvdeg.__version__)

Working on a  Windows 10
Python version  3.10.14 | packaged by Anaconda, Inc. | (main, Mar 21 2024, 16:20:14) [MSC v.1916 64 bit (AMD64)]
pvdeg version  0.2.4.dev50+gf2b8436.d20240401


## Define Single Point Scenario Object
Scenario is a general class that can be used to replace the legacy functional pvdeg analysis approach with an object orented one. ``Scenario`` can preform single location or geospatial analysis. The scenario constructor takes many arguments but the only required one for the following use cases is the ``name`` attribute. It is visible in when we display the entire scenario and is present in the file of saved information about the scenario. We also need to provide the class constructor with our API key and email.

A way around this is to provide the weather and metadata in the pipeline job arguments or you can load data from somewhere else and provide it in the same fashion.

<div class="alert alert-block alert-info">
<b>Please use your own API key: The block below makes an NSRDB API to get weather and meta data. This tutorial will work with the DEMO Key provided, but it will take you less than 3 minutes to obtain your own at <a ref="https://developer.nrel.gov/signup/">https://developer.nrel.gov/signup/</a> so register now.) 
</div>

In [3]:
simple_scenario = pvdeg.scenario.Scenario(
    name="Point Minimum Standoff",
    email='user@mail.com',
    api_key='DEMO_KEY'
)

## Adding A Location
To add a single point using data from the Physical Solar Model (PSM3), simply feed the scenario a single coordinate in tuple form via the ``addLocation`` method. ``see_added`` is a flag that allows you to see the new location added to the scenario. Currently this is the only way to add a location to a non-geospatial scenario, all of the other arguments are unusable when ``Scenario.geospatial == False``. 

Attempting to add a second location by calling the method again with a different coordinate pair will overwrite the old location data stored in the class instance. 

In [4]:
simple_scenario.addLocation(
    lat_long=(25.783388, -80.189029), 
    see_added=True,
)

Column "relative_humidity" not found in DataFrame. Calculating...


C:\Users\tford\dev\PVDegradationTools\pvdeg\scenario.py:290: UserWarning: Gids Added - [1060699]
  warnings.warn(message, UserWarning)


## Scenario Pipelines

The pipeline is a list of tasks called jobs for the scenario to run. We will populate the pipeline with a list of jobs before executing them all at once. 

To add a job to the pipeline use the ``updatePipeline`` method. Two examples of adding functions to the pipeline will be shown below.

### Adding a job without function arguments

The simplest case of adding a job to the pipeline is when it only requires us to provide simple weather and metadata. In the function definition and docstring these appear as ``weather_df`` and ``meta``. Since these attributes are contained in our scenario class instance we do not have to worry about them. We can simply add the function as shown below.

In [5]:
simple_scenario.addJob(
    func=pvdeg.standards.standoff,
    see_added=True
)

C:\Users\tford\dev\PVDegradationTools\pvdeg\scenario.py:503: UserWarning: standoff added to pipeline as 
 {'job': <function standoff at 0x0000020A31600700>, 'params': {}}
  warnings.warn(message, UserWarning)


### Adding a job with function arguments

When adding a job that contains a function requiring other arguments such as ``solder_fatigue`` which requires a value for ``wind_factor``, we will need to provide it. The most straightforeward way to do this is using a kwargs dictionary and passing it to the function. We do not unpack the dictionary before passing it. This is done inside of the scenario at pipeline runtime (when ``runPipeline`` is called).

In [6]:
kwargs = {'wind_factor' : 0.33}

simple_scenario.addJob(
    func=pvdeg.fatigue.solder_fatigue, 
    func_params=kwargs,
    see_added=True
)

C:\Users\tford\dev\PVDegradationTools\pvdeg\scenario.py:503: UserWarning: solder_fatigue added to pipeline as 
 {'job': <function solder_fatigue at 0x0000020A315E3370>, 'params': {'wind_factor': 0.33}}
  warnings.warn(message, UserWarning)


### Adding a job with weather and metadata from outside of the class
### Not functional 

could just directly set weather data with scenario.weather_data = weather and scenario.meta_data = meta but that would only work for all of the jobs in the pipeline

Say local weather data is available or other, if we want to use this rather than the PSM3 data at a latitude and longitude we can also provide the weather and metadata in the function arguments. This is probably the best if avoided but follows the same syntax as providing other function arguments. See the example below.

In [7]:
PSM_FILE = os.path.join(pvdeg.DATA_DIR,'psm3_demo.csv')
weather, meta = pvdeg.weather.read(PSM_FILE,'psm')

kwargs = {'weather_df' : weather, 'meta' : meta}

simple_scenario.addJob(
    func=pvdeg.standards.standoff,
    func_params=kwargs,
    see_added=True
)

# FIX THIS CASE IN SCENARIO CLASS
# (simple_scenario.pipeline[1]['job'])(**simple_scenario.pipeline[1]['params'])

C:\Users\tford\dev\PVDegradationTools\pvdeg\scenario.py:503: UserWarning: standoff added to pipeline as 
 {'job': <function standoff at 0x0000020A31600700>, 'params': {'weather_df':                            Year  Month  Day  Hour  Minute  dni  dhi  ghi  \
1999-01-01 00:30:00-07:00  1999      1    1     0      30  0.0  0.0  0.0   
1999-01-01 01:30:00-07:00  1999      1    1     1      30  0.0  0.0  0.0   
1999-01-01 02:30:00-07:00  1999      1    1     2      30  0.0  0.0  0.0   
1999-01-01 03:30:00-07:00  1999      1    1     3      30  0.0  0.0  0.0   
1999-01-01 04:30:00-07:00  1999      1    1     4      30  0.0  0.0  0.0   
...                         ...    ...  ...   ...     ...  ...  ...  ...   
1999-12-31 19:30:00-07:00  1999     12   31    19      30  0.0  0.0  0.0   
1999-12-31 20:30:00-07:00  1999     12   31    20      30  0.0  0.0  0.0   
1999-12-31 21:30:00-07:00  1999     12   31    21      30  0.0  0.0  0.0   
1999-12-31 22:30:00-07:00  1999     12   31    22      30 

## View Scenario

The ``viewScenario`` method provides an overview of the information contained within your scenario object. Here you can see if it contains the location weather and metadata. As well as the jobs in the pipeline and their arguments.

In [8]:
simple_scenario.viewScenario()

Name : Point Minimum Standoff
Pipeline : 
                                               job  params.wind_factor                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

## Executing Pipeline Jobs
To run the pipeline after we have populated it with the desired jobs call the ``runPipeline`` method on our scenario instance. This will run all of the jobs we have previously added. The functions that need weather and metadata will grab it from the scenario instance using the correct location added above. The pipeline jobs results will be saved to the scenario instance.

In [9]:
simple_scenario.run()

The array tilt angle was not provided, therefore the latitude tilt of 25.8 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
The array tilt angle was not provided, therefore the latitude tilt of 25.8 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.


The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
results_dict dtype : <class 'pandas.core.frame.DataFrame'>
{'standoff':          x     T98_0    T98_inf
0  1.20347  74.00243  50.322188, 'solder_fatigue': 4.562891779210449, 'standoff_2242801305344':           x      T98_0    T98_inf
0  2.008636  77.038644  50.561112}
results_dict dtype : <class 'numpy.float64'>
{'standoff':          x     T98_0    T98_inf
0  1.20347  74.00243  50.322188, 'solder_fatigue': 4.562891779210449, 'standoff_2242801305344':           x      T98_0    T98_inf
0  2.008636  77.038644  50.561112}
results_dict dtype : <class 'pandas.core.frame.DataFrame'>
{'standoff':          x     T98_0    T98_inf
0  1.20347  74.00243  50.322188, 'solder_fatigue': 4.562891779210449, 'standoff_2242801305344':           x      T98_0    T98_inf
0  2.008636  77.038644  50.561112}


## Results Series ##
We will use a series to store the various return values of functions run in our pipeline. These can partially obfuscate the dataframes within them so to access the dataframes, use the function name to access it

In [12]:
print(simple_scenario.results)
print("We can't see out data in here so we need to do another step", end="\n\n")

# to see all available ouputs of results do
print(f"this is the list of all available frames in results : {simple_scenario.results.index}")

display(simple_scenario.results['standoff'])
display(simple_scenario.results['standoff_2242801305344'])
display(simple_scenario.results['solder_fatigue'])

standoff                           x     T98_0    T98_inf
0  1.20347  74...
solder_fatigue                             solder_fatigue
0        4.562892
standoff_2242801305344              x      T98_0    T98_inf
0  2.008636 ...
dtype: object
We can't see out data in here so we need to do another step

this is the list of all available frames in results : Index(['standoff', 'solder_fatigue', 'standoff_2242801305344'], dtype='object')


,x,T98_0,T98_inf
0,1.20347,74.00243,50.322188


,x,T98_0,T98_inf
0,2.008636,77.038644,50.561112


,solder_fatigue
0,4.562892


## Cleaning Up the Scenario

Each scenario object creates a directory named ``pvd_job_...`` that contains information about the scenario instance. To remove the directory and all of its information call ``clean`` on the scenario. This will permanently delete the directory created by the scenario.

In [13]:
simple_scenario.clean()